In [ ]:
import cv2
import numpy as np
import os
from utils import *
import matplotlib.pyplot as plt

In [ ]:
#Medir contraste
def calcular_varianza_histograma(img_gray):
    """
    Calcula la varianza en el histograma para saber el contraste de la imagen 
    Retorna:
        - Varianza (dispersión de intensidades).
    """
       
    # Calcular histograma
    hist = cv2.calcHist([img_gray], [0], None, [256], [0, 256]).flatten()
    
    total_pixeles = img_gray.size
    valores = np.arange(256) # Valores posibles de intensidad (0-255)
    media = np.sum(valores * hist) / total_pixeles
    varianza = np.sum(((valores - media) ** 2) * hist) / total_pixeles

    return hist, varianza

In [ ]:
def procesar_imagenes_carpeta_contraste(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas"
):
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)

        if not os.path.isdir(ruta_subcarpeta):
            continue

        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta, "contraste")
        os.makedirs(carpeta_salida_sub, exist_ok=True)

        ruta_txt = os.path.join(carpeta_salida_sub, "info_varianzas_contraste.txt")
        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Varianzas de los histogramas en escala de grises de las imágenes\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("La varianza indica la dispersión de los niveles de gris.\n")
            archivo_txt.write("Valores mayores de la varianza corresponden a mayor contraste.\n\n")

            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if not filename.lower().endswith(".png"):
                    continue

                ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                img_bgr = cv2.imread(ruta_imagen_entrada)
                if img_bgr is None:
                    continue

                img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

                # Calcular la varianza del histograma
                hist, varianza = calcular_varianza_histograma(img_gray)

                nombre, _ = os.path.splitext(filename)
                nombre_salida = f"{nombre}_histograma_grises.png"
                ruta_imagen_salida = os.path.join(carpeta_salida_sub, nombre_salida)

                # Graficar y guardar el histograma
                plt.figure()
                plt.plot(hist, color='black')
                plt.title(f"Histograma de {filename}")
                plt.xlabel("Intensidad de píxeles")
                plt.ylabel("Frecuencia")
                plt.savefig(ruta_imagen_salida)
                plt.close()
                
                archivo_txt.write(f"{filename} -> varianza: {varianza:.3f}\n")

    print("Procesamiento completado con cálculo de contraste.")

In [ ]:
procesar_imagenes_carpeta_contraste(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas"
)